## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup darknet environment

In [ ]:
# check whether GPU is provided
!nvcc --version

In [ ]:
import os
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# remove the existing folder if have
#!rm -r darknet_for_colab

# download and compile darknet_for_colab
!git clone https://github.com/quangnhat185/darknet_for_colab.git
%cd darknet_for_colab
!make
!chmod +x ./darknet

## Download yolov4 pre-trained weights
This is not necessary if you are continuing the training with your weights.

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## Getting images from Google Drive

In [ ]:
os.chdir("/content/drive/My Drive")

!cp dataset4.zip /content/darknet_for_colab/data

os.chdir("/content/darknet_for_colab")

In [ ]:
%cd data
assert os.getcwd()=='/content/darknet_for_colab/data', 'Directory should be "/content/darknet_for_colab/data" instead of "{}"'.format(os.getcwd())

# download custom data of common traffic signs
#!wget --no-check-certificate "https://onedrive.live.com/download?cid=A86CBC7F31A1C06B&resid=A86CBC7F31A1C06B%21121&authkey=AMUUk0Np4tqH3n4" -O ts.zip
!unzip dataset4.zip
!rm -f dataset4.zip
!ls
%cd ..


## Visualize custom dataset examples (optional)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob

def read_label(image_path):
  file_name = image_path.replace('.jpg', '.txt')
  with open(file_name, 'rt') as file:
    print(os.path.basename(file_name) + ': \n' + file.read())

image_path = glob.glob("data/ts/*.jpg")
fig = plt.figure(figsize=(12,8))
cols = 2
rows = 2
grid = gridspec.GridSpec(nrows=rows, ncols=cols, figure=fig)
for i in range(cols*rows):
  fig.add_subplot(grid[i])
  image=plt.imread(image_path[i])
  plt.title(os.path.basename(image_path[i]))
  plt.axis(False)
  plt.imshow(image)
  read_label(image_path[i])

plt.savefig("dataset_examples.jpg", dpi=300)

## Modify yolov4 architecture


##!! yolov4_config.py and yolov4.data files should be changed according to your setup. For example I have changed class number to 6.


In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Run python script to create our customize yolov4_custom_train.cfg 
# and yolov4_custom_tes.cfg in folder /cfg
!python yolov4_setup.py

## Create symbolic link in Drive

Backup folder in darknet gets removed during training, so we have to create a backup folder in our drive and move it here.

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

!rm -r backup

os.chdir("/content/drive/My Drive")

## /content/drive/My Drive klasöründe el ile backup dosyası oluştur.
!cp -r backup /content/darknet_for_colab

os.chdir("/content/darknet_for_colab")

Getting my last trained weights from the drive.


In [ ]:
os.chdir("/content/drive/My Drive")

!cp yolov4_custom_train_last.weights /content/darknet_for_colab

os.chdir("/content/darknet_for_colab")

## Training 

If you are training from scratch, change weights to yolov4.conv.137

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg yolov4_custom_train_last.weights -dont_show -map
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above

## Visualize training result

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Plotting training result after 2000 epochs
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
train_result = plt.imread("chart.png")
plt.axis(False)
plt.imshow(train_result)